# Teste pyomo

In [1]:
import numpy as np
from udgp import Instance
from udgp import M2

In [27]:
instance = Instance.artificial_molecule(5, freq=True, seed=1234567)
instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [28]:
instance.reset()
model = M2(instance, nx=4, ny=1)

In [29]:
model.optimize(solver="gurobi", log=True)

        0.03 seconds required for presolve
Set parameter NonConvex to value 2
Set parameter MIPGapAbs to value 0.05
Set parameter IntFeasTol to value 0.0001
Set parameter FeasibilityTol to value 0.0001
Set parameter OptimalityTol to value 0.0001
Set parameter Cuts to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 364 rows, 212 columns and 958 nonzeros
Model fingerprint: 0xfda98b40
Model has 10 quadratic constraints
Variable types: 172 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-03, 3e+00]
  RHS range        [6e-01, 7e+00]
Presolve removed 72 rows and 22 columns
Presolve time: 0.00s
Presolved: 422 rows, 230 columns, 1224 nonzeros
Presolved model h

True

In [30]:
print(instance.is_solved())
instance.view()

True


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol